## Setup

1. Complete the previous challenges.

In [13]:
# Azure Storage account settings:
azure_storage_account = 'YOUR_STORAGE_ACCOUNT'
blob_connection_string = "YOUR_BLOB_STORE_CONNECTION_STRING"
blob_container = "YOUR_BLOB_CONTAINER"

# Azure Search constants
azsearch_url = "YOUR_SEARCH_SERVICE.search.windows.net"
azsearch_key = "YOUR_SEARCH_KEY"

# Cognitive Services Key
cog_svc_key = "YOUR_COGNITIVE_SERVICE_KEY"

# Function URL
function_url = 'YOUR_FUNCTION_URL'

## Function to wrap HTTP requests

In [14]:
def azsearch_rest(request_type="GET", function_name="servicestats", body=None):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }

    # Request parameters
    params = urllib.parse.urlencode({
        'api-version':'2019-05-06-Preview'
    })
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/{0}?{1}".format(function_name, params)
        conn.request(request_type, request_path, body, headers)
        response = conn.getresponse()
        print(response.status)
        data = response.read().decode("UTF-8")
        result = None
        if len(data) > 0:
            result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API")

Ready to use the REST API


### Create a Synonym Map

In [15]:
import json

body = {
   "format":"solr",
   "synonyms": "USA, United States, America, United States of America\n" \
      "UK, GB, United Kingdom, Great Britain, Britain\n" \
      "UAE, United Arab Emirates, Emirates\n"
}

try:
    result = azsearch_rest(request_type="PUT", function_name="synonymmaps/locationsynonyms", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)
    

204


## Update the Index

In [16]:
import json

# Define the request body
body = {
  "fields": [
    {
      # Unique ID for each document
      "name": "id",
      "type": "Edm.String",
      "key": True,
      "searchable": True,
      "filterable": True,
      "facetable": False,
      "sortable": True
    },
    {
     # The file path of the document
      "name": "url",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The file name of the document
      "name": "file_name",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The size of the document
      "name": "size",
      "type": "Edm.Int64",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The last-modified date of the document
      "name": "last_modified",
      "type": "Edm.DateTimeOffset",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The text content in the document
      "name": "content",
      "type": "Edm.String",
      "sortable": False,
      "searchable": True,
      "filterable": True,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    },
    {
      # The sentiment score
      "name": "sentiment",
      "type": "Edm.Double",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": True
    },
    {
      # Key phrases that might help identify the subject of the text
      "name": "key_phrases",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "filterable": False,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    },
    {
      # Geographical locations mentioned in the text
      "name": "locations",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    },
    {
      # People mentioned in the text
      "name": "people",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # URLs mentioned in the text
      "name": "links",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # URLs mentioned in the text
      "name": "entities",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },    {
      # The image content in the document
      "name": "image_descriptions",
      "type": "Collection(Edm.String)",
      "sortable": False,
      "searchable": True,
      "filterable": True,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    },
    {
      # The text contained in images in the document
      "name": "image_text",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    },
    {
      # The combined source text plus the text extracted from images
      "name": "merged_text",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": False,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    },
    {
      # The most common words in the text
      "name": "top_words",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False,
      "synonymMaps":["locationsynonyms"]
    }
  ],
    "suggesters": [
    {
      "name": "location_suggester",
      "searchMode": "analyzingInfixMatching",
      "sourceFields": ["locations"]
    }
  ],
  "scoringProfiles": [  
    {  
      "name": "boost_relevance",  
      "text": {  
        "weights": {  
          "key_phrases": 5.0 
        }
      },
          "functions": [  
            {  
              "type": "freshness",  
              "fieldName": "last_modified",  
              "boost": 2,  
              "interpolation": "quadratic",  
              "freshness": {  
                "boostingDuration": "P365D"  
              }  
            },  
            {
              "type": "magnitude",  
              "fieldName": "size",  
              "boost": 2,  
              "interpolation": "linear",  
              "magnitude": {  
                "boostingRangeStart": 1000,  
                "boostingRangeEnd": 500000,  
                "constantBoostBeyondRange": "true"
              }  
            }
          ]
      }
  ]
}

try:
    result = azsearch_rest(request_type="PUT", function_name="indexes/website-docs-index", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)
    

204


## Rerun the Indexer

In [17]:
import time

try:
    # Reset the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/website-docs-indexer/reset", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
        
    # re-run the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/website-docs-indexer/run", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
    


except Exception as e:
    print('Error:')
    print(e)

204
202


In [18]:
import time, json

try:
    complete = False
    while (complete == False):
        result = azsearch_rest(request_type="GET", function_name="indexers/website-docs-indexer/status")
        state = result["status"]
        if result['lastResult'] is not None:
            state = result['lastResult']['status']
        print (state)
        if state in ("success", "error"):
            complete = True
        time.sleep(1)

except Exception as e:
    print('Error:')
    print(e)

200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
success


## Create Wrapper Function for Query Requests

In [19]:
def azsearch_query(index, params):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/indexes/{0}/docs?{1}".format(index, params)
        conn.request("GET", request_path, None, headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API for Queries")

Ready to use the REST API for Queries


## Queries

### Search for Country

In [20]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    '$count':'true',
    '$select':'url, file_name, merged_text',
    'api-version':'2019-05-06-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"])
        print(value["merged_text"])

except Exception as e:
    print('Error:')
    print(e)

Search: Britain
Hits: 19

 https://margiesstore.blob.core.windows.net/margies-data/website_docs/collateral/London%20Brochure.pdf
London Brochure.pdf

Margie’s Travel Presents… 

London 
London is the capital and 

most populous city of 

England and the United 

Kingdom. Standing on the 

River Thames in the south 

east of the island of Great 

Britain, London has been 

a major settlement for two 

millennia. It was founded 

by the Romans, who 

named it Londinium. 

London's ancient core, the 

City of London, largely 

retains its 1.12-square-

mile medieval boundaries. 

Since at least the 19th century, London has also referred to the metropolis around this core, 

historically split between Middlesex, Essex, Surrey, Kent, and Hertfordshire, which today largely 

makes up Greater London, governed by the Mayor of London and the London Assembly. 

 

Mostly popular for: 
Leisure, Outdoors, Historical, Arts 
& Culture 
 
Best time to visit: 
Jun-Aug  
Averag Precipitation: 1.9 in 
A

### Suggestions

In [21]:
def azsearch_suggest(index, params):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/indexes/{0}/docs/suggest?{1}".format(index, params)
        conn.request("GET", request_path, None, headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API for Suggestions")

Ready to use the REST API for Suggestions


In [22]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'{0}'.format(search_terms),
    'suggesterName':'location_suggester',
    'fuzzy':'false',
    'highlightPreTag':'[',
    'highlightPostTag':']',
    'api-version':'2019-05-06-Preview'
})

try:
    result = azsearch_suggest(index="website-docs-index", params=searchParams)
    unique_terms = []
    for value in result["value"]:
        one_line = value["@search.text"].replace("\n","")
        if(one_line not in unique_terms):
            unique_terms.append(one_line);
            print(one_line)

except Exception as e:
    print('Error:')
    print(e)

Search: uni
[Uni]ted Kingdom
[Uni]ted States
headquarters of the [Uni]ted Nations
[Uni]ted


### Autocomplete

In [23]:
def azsearch_autocomplete(index, params):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/indexes/{0}/docs/autocomplete?{1}".format(index, params)
        conn.request("GET", request_path, None, headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API for Autocompletion")

Ready to use the REST API for Autocompletion


In [24]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'{0}'.format(search_terms),
    'suggesterName':'location_suggester',
    'autocompleteMode':'twoTerms',
    'api-version':'2019-05-06-Preview'
})

try:
    result = azsearch_autocomplete(index="website-docs-index", params=searchParams)
    for value in result["value"]:
        print(value['text'])
except Exception as e:
    print('Error:')
    print(e)

Search: San
san francisco
san diego
san jose


### Scoring

#### Default Scoring

In [11]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    '$count':'true',
    '$select':'url, file_name, size, key_phrases, last_modified',
    '$filter': "search.ismatch('reviews', 'url')",
    'api-version':'2019-05-06-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\nScore:",value["@search.score"])
        print(value["url"])
        print(value["size"], value["last_modified"])
        print(value["key_phrases"])

except Exception as e:
    print('Error:')
    print(e)

Search: quiet
Hits: 9

Score: 0.21962124
https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201802.pdf
112199 2019-07-26T16:50:51Z
['London', 'Buckingham Hotel', 'quietest hotel', 'quiet stay', 'addition good sized bathroom']

Score: 0.115804866
https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201835.pdf
112533 2019-07-26T16:50:46Z
['staff', 'Lombard Hotel', 'San Francisco', 'USA', 'wonderful trip']

Score: 0.07515595
https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201823.pdf
107762 2019-07-26T16:50:48Z
['London', 'City Hotel', 'accommodation excellent', 'UK', 'Reviewer']

Score: 0.06772463
https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201822.pdf
112519 2019-07-26T16:50:50Z
['Awesome hotel', 'City Hotel', 'fantastic hotel', 'water dam', 'huge tub']

Score: 0.06326851
https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201860.pdf
105730 2019-07-26T16

#### Boosted Scoring

In [12]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    'scoringProfile': 'boost_relevance',
    '$count':'true',
    '$select':'url, file_name, size, key_phrases, last_modified',
    '$filter': "search.ismatch('reviews', 'url')",
    'api-version':'2019-05-06-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\nScore:",value["@search.score"])
        print(value["url"])
        print(value["size"], value["last_modified"])
        print(value["key_phrases"])

except Exception as e:
    print('Error:')
    print(e)

Search: quiet
Hits: 9

Score: 1.757307
https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201802.pdf
112199 2019-07-26T16:50:51Z
['London', 'Buckingham Hotel', 'quietest hotel', 'quiet stay', 'addition good sized bathroom']

Score: 0.21670124
https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201835.pdf
112533 2019-07-26T16:50:46Z
['staff', 'Lombard Hotel', 'San Francisco', 'USA', 'wonderful trip']

Score: 0.14390178
https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201822.pdf
112519 2019-07-26T16:50:50Z
['Awesome hotel', 'City Hotel', 'fantastic hotel', 'water dam', 'huge tub']

Score: 0.13803802
https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201823.pdf
107762 2019-07-26T16:50:48Z
['London', 'City Hotel', 'accommodation excellent', 'UK', 'Reviewer']

Score: 0.11749724
https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201860.pdf
105730 2019-07-26T16:50

## LUIS

In [25]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

LUIS_KEY = 'YOUR LUIS KEY'

version_id = "1.0"

# Create LUIS App
body = {
    "name": "MargieSearch",
    "description": "search query interpreter",
    "culture": "en-us",
    "initialVersionId": version_id
}

headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': LUIS_KEY,
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/luis/api/v2.0/apps/?%s" % params, str(body), headers)
    response = conn.getresponse()
    app_id = response.read().decode("UTF-8")
    print(app_id)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

"6635a47a-ab9a-4dfc-841a-a9f4d98d9ac8"


In [26]:
app_id = app_id.replace('"', '')

In [27]:
body = {"name": "GetReviews"}

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/luis/api/v2.0/apps/" + app_id + "/versions/" + version_id + "/intents?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'"18429974-6415-4342-b75d-16208bbff0e1"'


In [28]:
body = {"name": "GetDocs"}

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/luis/api/v2.0/apps/" + app_id + "/versions/" + version_id + "/intents?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'"adf4c857-659c-42ba-a213-779de56e3b3b"'


In [29]:
body = {"name": "destination"}

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/luis/api/v2.0/apps/" + app_id + "/versions/" + version_id + "/entities?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'"fb124c07-d04a-4655-ae63-71e10bfbbe4f"'


In [30]:
body = {
    "text": "Get reviews for hotels in New York",
    "intentName": "GetReviews",
    "entityLabels":
    [
        {
            "entityName": "destination",
            "startCharIndex": 26,
            "endCharIndex": 33
        }
    ]
}

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/luis/api/v2.0/apps/" + app_id + "/versions/" + version_id + "/example?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'{"ExampleId":494086734,"UtteranceText":"get reviews for hotels in new york"}'


In [31]:
body = {
    "text": "Get reviews for hotels in London",
    "intentName": "GetReviews",
    "entityLabels":
    [
        {
            "entityName": "destination",
            "startCharIndex": 26,
            "endCharIndex": 31
        }
    ]
}

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/luis/api/v2.0/apps/" + app_id + "/versions/" + version_id + "/example?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'{"ExampleId":494087836,"UtteranceText":"get reviews for hotels in london"}'


In [32]:
body = {
    "text": "What do people say about hotels in London",
    "intentName": "GetReviews",
    "entityLabels":
    [
        {
            "entityName": "destination",
            "startCharIndex": 35,
            "endCharIndex": 40
        }
    ]
}

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/luis/api/v2.0/apps/" + app_id + "/versions/" + version_id + "/example?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'{"ExampleId":494088278,"UtteranceText":"what do people say about hotels in london"}'


In [33]:
body = {
    "text": "Get information about London",
    "intentName": "GetDocs",
    "entityLabels":
    [
        {
            "entityName": "destination",
            "startCharIndex": 22,
            "endCharIndex": 27
        }
    ]
}

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/luis/api/v2.0/apps/" + app_id + "/versions/" + version_id + "/example?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'{"ExampleId":494088280,"UtteranceText":"get information about london"}'


In [34]:
body = {
    "text": "Tell me about Las Vegas",
    "intentName": "GetDocs",
    "entityLabels":
    [
        {
            "entityName": "destination",
            "startCharIndex": 14,
            "endCharIndex": 22
        }
    ]
}

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/luis/api/v2.0/apps/" + app_id + "/versions/" + version_id + "/example?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'{"ExampleId":494090464,"UtteranceText":"tell me about las vegas"}'


In [38]:
body = {
    "text": "Get information about London",
    "intentName": "GetDocs",
    "entityLabels":
    [
        {
            "entityName": "destination",
            "startCharIndex": 22,
            "endCharIndex": 27
        }
    ]
}

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/luis/api/v2.0/apps/" + app_id + "/versions/" + version_id + "/example?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'{"ExampleId":494088280,"UtteranceText":"get information about london"}'


In [35]:
try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/luis/api/v2.0/apps/" + app_id + "/versions/" + version_id + "/train?%s" % params, None, headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))


b'{"statusId":9,"status":"Queued"}'


In [36]:
try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("GET", "/luis/api/v2.0/apps/" + app_id + "/versions/" + version_id + "/train?%s" % params, None, headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'[{"modelId":"adf4c857-659c-42ba-a213-779de56e3b3b","details":{"statusId":0,"status":"Success","exampleCount":5,"trainingDateTime":"2019-07-26T18:05:09Z"}},{"modelId":"18429974-6415-4342-b75d-16208bbff0e1","details":{"statusId":0,"status":"Success","exampleCount":5,"trainingDateTime":"2019-07-26T18:05:09Z"}},{"modelId":"9378012d-f373-4099-a811-52aac63838ce","details":{"statusId":0,"status":"Success","exampleCount":5,"trainingDateTime":"2019-07-26T18:05:09Z"}},{"modelId":"fb124c07-d04a-4655-ae63-71e10bfbbe4f","details":{"statusId":0,"status":"Success","exampleCount":5,"trainingDateTime":"2019-07-26T18:05:09Z"}}]'


In [37]:
body = {
   "versionId": "1.0",
   "isStaging": "false"
}

try:
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/luis/api/v2.0/apps/" + app_id + "/publish?%s" % params, str(body), headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'{"versionId":"1.0","directVersionPublish":false,"endpointUrl":"https://westus.api.cognitive.microsoft.com/luis/v2.0/apps/6635a47a-ab9a-4dfc-841a-a9f4d98d9ac8","isStaging":false,"assignedEndpointKey":null,"region":"westus","endpointRegion":"westus","publishedDateTime":"2019-07-26T18:06:14Z","failedRegions":""}'


In [39]:
def azsearch_query(index, params):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/indexes/{0}/docs?{1}".format(index, params)
        conn.request("GET", request_path, None, headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API for Queries")

Ready to use the REST API for Queries


In [40]:
!pip install azure-cognitiveservices-language-luis

     |████████████████████████████████| 245kB 2.2MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [41]:
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials


# Set up API configuration
luis_region = "westus"

luis_endpoint = "https://" + luis_region + ".api.cognitive.microsoft.com"

luis_credentials = CognitiveServicesCredentials(LUIS_KEY)

# prompt for a command
command = input('Please enter a command: \n')

luis_client = LUISRuntimeClient(luis_endpoint, luis_credentials)
data = luis_client.prediction.resolve(app_id, command)
intent = data.top_scoring_intent.intent
destination = data.entities[0].entity
print (intent, destination)

search_terms = destination
if intent == "GetReviews":
    search_terms = search_terms + " + url:reviews"
    
# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'{0}'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    '$count':'true',
    '$select':'url, file_name, merged_text',
    'api-version':'2019-05-06-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"])
        print(value["merged_text"])

except Exception as e:
    print('Error:')
    print(e)

Please enter a command: 
What do people say about hotels in Las Vegas?
GetReviews las vegas
Hits: 13

 https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201857.pdf
201857.pdf

An amazing experience 

The Canal Hotel, Las Vegas, USA 

11/22/2018 

I expected something over the top but this was even more, just the check in alone is so ornate, gold 

everywhere, mural ceilings, and all rooms are suites with nothing spared, it is also positioned in a 

great spot on the strip 

Reviewer: Rashmi Dixit 




 https://margiesstore.blob.core.windows.net/margies-data/website_docs/reviews/201848.pdf
201848.pdf

Does not live up to its reputation! 

The Fountain Hotel, Las Vegas, USA 

11/20/2018 

First impressions on check in are of how enormous and busy the hotel is - checked into our room 

which was dark and dreary, with a view of a massive building site - not quite what we were 

expecting - when we requested to change rooms we were told that there was no other roo